<a href="https://colab.research.google.com/github/papichoolo/healthchatbot/blob/main/Medichatbot_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
dataset=load_dataset("sid6i7/patient-doctor",split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['Unnamed: 0', 'Description', 'Patient', 'Doctor', 'problem_description'],
    num_rows: 106220
})

In [4]:
dataset.info

DatasetInfo(description='', citation='', homepage='', license='', features={'Unnamed: 0': Value(dtype='int64', id=None), 'Description': Value(dtype='string', id=None), 'Patient': Value(dtype='string', id=None), 'Doctor': Value(dtype='string', id=None), 'problem_description': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='csv', dataset_name='patient-doctor', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=163901523, num_examples=106220, shard_lengths=None, dataset_name='patient-doctor')}, download_checksums={'hf://datasets/sid6i7/patient-doctor@e19a01da8255fa52987d2339321bcb4036751c5f/data.csv': {'num_bytes': 162940981, 'checksum': None}}, download_size=162940981, post_processing_size=None, dataset_size=163901523, size_in_bytes=326842504)

In [5]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [31]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [34]:
notebook_login()

Loading Model

In [48]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda:0",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [50]:
device="cuda:0"

Setting LoRA Config

In [51]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

Generating Prompt based on Dataset

In [52]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Patient"]}
<assistant>: {data_point["Doctor"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [53]:
data = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/106220 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Time to train POGGERS


In [54]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.240800
2,3.467700
3,3.222500
4,2.833200
5,2.937900
6,3.346900
7,3.377700
8,3.504900
9,3.394500
10,2.886400


KeyboardInterrupt: ignored